In [ ]:
import numpy as np
import tensorflow as tf
#tf.enable_eager_execution()
import tensorflow.keras as keras
from TicTacToeSetup import isStraitConnection, getVacationsIterator
from TernaryPNS import iterateBidirSearch, UNKNOWN
from TicTacToePNS import RenderNode, getTicTacToeRootNode, flip, TicTacToeState, \
    BidirPNSNode, TranspositionNode, PlaySetKey
from treeutil import XRenderTree
from itertools import filterfalse

In [ ]:
THRESHOLD = 11
node = root
while node.visit_num >= THRESHOLD:
    node = select(node)
    path.append(node)
    
R = playout(node)
for node in reversed(path):
    update(node,R)

def simulate(node):
    getActionsIter(node)
    np.argmax(np.random.multinomial(1, dist[0], size=10), axis=1)
    

In [ ]:
def makeMove(board, loc, player):
    b = board.copy()
    b[loc] = player
    return b

def simulate(board, player, sess):
    depth = np.sum(board != 0)
    p = player
    while True:
        mod = [-1.0, 1.0][player == p]
        if depth > 9:
            return mod * evalPNS(board, p)

        actions = list(getVacationsIterator(board))
        positions, is_win = zip(*[
            (makeMove(board, loc, p), isStraitConnection(board, loc, p))
            for loc in actions])
        if any(is_win):
            return mod
        
        eval_positions = positions
        if player != p:
            #positions = [np.vectorize(flip)(board) for board in positions]
            eval_positions = np.vectorize(flip)(positions)
        #dist_op = keras.activations.softmax(evaluate(eval_positions))
        dist = sess.run(keras.activations.softmax(evaluate(eval_positions)))[0]
        i = np.argmax(np.random.multinomial(1, dist))
        board = positions[i]
        depth += 1
        p = flip(p)

In [ ]:
def evaluate(positions):
    codes = np.array(list(map(augment, positions)), dtype=np.float32)
    shape = codes.shape
    codes = codes.reshape(1, -1, shape[-4], shape[-3]*shape[-2]*shape[-1])
    return model(codes)

# feature coding and data augmentation:
CODE_MSB_VALUE = 1<<3
def codeBoard(board):
    return np.array(list(map(
        lambda L:list(map(lambda l: [int(x) for x in bin((1<<l)+CODE_MSB_VALUE)[3:]], L)),
        board
    )))

def getRot3(board):
    rotations = [board]
    for i in range(3):
        rotations.append(np.rot90(rotations[-1]))
    return rotations

def augment(board):
    flip = np.transpose(board)
    return [codeBoard(v) for v in getRot3(board)+getRot3(flip)]

In [ ]:
class vModel(tf.keras.Model):
    def __init__(self):
        super(vModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(units=44, name='dense1')
        self.dense2 = tf.keras.layers.Dense(units=1, name='dense2')

    def call(self, input):
        x = self.dense1(input)
        x = tf.squeeze(self.dense2(x), axis=-1)
        return tf.math.reduce_max(x, axis=2)

model = vModel()

In [ ]:
test_board = np.array(
        [[0,1,1,0],
         [0,2,2,0],
         [0,2,2,0],
         [0,1,1,0]
        ], dtype = int)
flip_board = np.vectorize(flip)(test_board)
test_board_2 = np.array(
        [[0,0,0,0],
         [0,0,0,0],
         [0,0,0,0],
         [0,0,0,0]
        ], dtype = int)

In [ ]:
#np.random.seed(111)
player = 1
board = test_board_2
depth = np.sum(board != 0)
p = player
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    while True:
        mod = [-1.0, 1.0][player == p]
        if depth > 9:
            R = evalPNS(board, p)
            break

        actions = list(getVacationsIterator(board))
        positions, is_win = zip(*[
            (makeMove(board, loc, p), isStraitConnection(board, loc, p))
            for loc in actions])
        if any(is_win):
            R = 1.0
            break

        eval_positions = positions
        if player != p:
            #positions = [np.vectorize(flip)(board) for board in positions]
            eval_positions = np.vectorize(flip)(positions)
        #dist_op = keras.activations.softmax(evaluate(eval_positions))
        dist = sess.run(keras.activations.softmax(evaluate(eval_positions)))[0]
        i = np.argmax(np.random.multinomial(1, dist))
        board = positions[i]
        print(board)
        depth += 1
        p = flip(p)
    R *= mod
R

In [ ]:
player = 2
actions = list(getVacationsIterator(test_board))
positions, is_win = zip(*[
    (makeMove(test_board, loc, player), isStraitConnection(test_board, loc, player))
    for loc in actions])
e = evaluate(positions)
p = keras.activations.softmax(e)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    ev = sess.run(e)

In [ ]:
i = np.argmax(np.random.multinomial(1, dist))
i

In [ ]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    R = simulate(test_board_2, 1, sess)
R

In [ ]:
class rootState(TicTacToeState):
    def __init__(self, board, player):
        self.depth = np.sum(test_board != 0)
        self.board = board
        self.loc = (-1,-1)
        self.player = flip(player)
        TicTacToeState.initial_player = player
        self.key = PlaySetKey([])

    def evaluate(self):
        return UNKNOWN

def createRootNode(board, player):
    BidirPNSNode.count = 0
    root = TranspositionNode(rootState(board, player))
    TranspositionNode.table = {root.state.getKey():root}
    return root

def evalPNS(board, player):
    root = createRootNode(board, player)
    iterateBidirSearch(root)
    if root.proof_num == 0:
        return 1.0
    elif root.disproof_num == 0:
        return -1.0
    return 0.0

In [ ]:
evalPNS(flip_board, 2)

In [ ]:
root = createRootNode(flip_board, 1)
iterateBidirSearch(root)

In [ ]:
hash(root.state.getKey())

In [ ]:
print(XRenderTree(testRenderNode(root), 
                  childiter=lambda chld: filterfalse(lambda i: not i.orig.isExpanded() or i.orig.state.depth > 8, chld),
                  nodeiter=testRenderNode.citer))

In [ ]:
print(XRenderTree(testRenderNode(root), nodeiter=testRenderNode.citer))

In [ ]:
def evaluate(board, player):
    for 
    depth = np.sum(board != 0)
    if depth > 8:
        return evalPNS(board, player)
    

In [ ]:
class testRenderNode():
    def __init__(self, orig, parent=None):
        self.orig = orig
        if parent:
            parent_key_set = set(parent.orig.state.key.key_set)
        else:
            parent_key_set = set()
        self.loc = next(iter(set(orig.state.key.key_set) - parent_key_set),"root")

    def citer(self):
        return map(lambda orig: RenderNode(orig, self), self.orig.children)

    def __repr__(self):
        return str(self.loc)+"--"+str(self.orig)